

## PROJENİN AMACI:Bir teslimat şirketi için tahmine dayalı bakım modeli oluşturma -classification








 <img src="https://www.corporatevision-news.com/wp-content/uploads/2022/03/Shipping-Company.jpg" alt="Resim Açıklaması" width="670" height="400">



In [1]:
import pandas as pd                      #modelleri import ettik.
pd.set_option("display.max_columns",100) #en fazla 100 kolon göstermek ister isek.
import numpy as np

from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split   #verileri 2 ye bölmek için import ettik.

import warnings
warnings.filterwarnings('ignore')   #uyarıları görmemek için impor ettik

In [2]:
df=pd.read_csv('/content/failure.csv')

### EDA exploratory data analysis-keşifsel veri analizi
 <img src="https://cdn.prod.website-files.com/666bbba4ff7240a20f4cccf8/666bbba4ff7240a20f4ce4da_Data%20Exploration%20(1).png" alt="Resim Açıklaması" width="300" height="200">

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        124494 non-null  object
 1   device      124494 non-null  object
 2   failure     124494 non-null  int64 
 3   attribute1  124494 non-null  int64 
 4   attribute2  124494 non-null  int64 
 5   attribute3  124494 non-null  int64 
 6   attribute4  124494 non-null  int64 
 7   attribute5  124494 non-null  int64 
 8   attribute6  124494 non-null  int64 
 9   attribute7  124494 non-null  int64 
 10  attribute8  124494 non-null  int64 
 11  attribute9  124494 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 11.4+ MB


In [4]:
df.head(3)

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0


In [5]:
df.isnull().sum()

,0
date,0
device,0
failure,0
attribute1,0
attribute2,0
attribute3,0
attribute4,0
attribute5,0
attribute6,0
attribute7,0


In [6]:
df['date']

,date
0,2015-01-01
1,2015-01-01
2,2015-01-01
3,2015-01-01
4,2015-01-01
...,...
124489,2015-11-02
124490,2015-11-02
124491,2015-11-02
124492,2015-11-02


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
df['date'] = pd.to_datetime(df['date'])


In [9]:
# Yeni öznitelikler çıkarma
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [10]:
df.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,year,month,day
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7,2015,1,1
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0,2015,1,1
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0,2015,1,1
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0,2015,1,1
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3,2015,1,1


In [11]:
df['device'] = df['device'].str[4:]
#baştaki kısım aynı olduğu için okısmı çıkardım

In [12]:
df2=df.groupby('device')

In [13]:
df2.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,year,month,day
0,2015-01-01,1085,0,215630672,56,0,52,6,407438,0,0,7,2015,1,1
1,2015-01-01,166B,0,61370680,0,3,0,6,403174,0,0,0,2015,1,1
2,2015-01-01,1E6Y,0,173295968,0,0,0,12,237394,0,0,0,2015,1,1
3,2015-01-01,1JE0,0,79694024,0,0,0,6,410186,0,0,0,2015,1,1
4,2015-01-01,1R2B,0,135970480,0,0,0,15,313173,0,0,3,2015,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79390,2015-05-09,3499,0,215513480,0,0,0,8,684048,0,0,0,2015,5,9
79790,2015-05-10,13BB,0,85127128,0,0,0,5,689062,0,0,0,2015,5,10
79793,2015-05-10,2W1L,0,135713032,0,0,0,8,660032,0,0,0,2015,5,10
79794,2015-05-10,2XLX,0,228491848,0,0,0,8,687802,0,0,0,2015,5,10


In [14]:
df['failure'].value_counts() #verilerde dengesizlik olduğu için smote kullanacağız

,count
failure,
0,124388
1,106


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        124494 non-null  datetime64[ns]
 1   device      124494 non-null  object        
 2   failure     124494 non-null  int64         
 3   attribute1  124494 non-null  int64         
 4   attribute2  124494 non-null  int64         
 5   attribute3  124494 non-null  int64         
 6   attribute4  124494 non-null  int64         
 7   attribute5  124494 non-null  int64         
 8   attribute6  124494 non-null  int64         
 9   attribute7  124494 non-null  int64         
 10  attribute8  124494 non-null  int64         
 11  attribute9  124494 non-null  int64         
 12  year        124494 non-null  int32         
 13  month       124494 non-null  int32         
 14  day         124494 non-null  int32         
dtypes: datetime64[ns](1), int32(3), int64(10), object(1

### **DATA** **Visualization**

In [21]:
import plotly.express as px

# Dağılım grafiği oluşturma

fig = px.scatter(df, x='attribute6', y='month', color='failure', title='Scatter Plot of Age vs Attribute by Device')

# Grafik düzenleme
fig.update_layout(
    xaxis_title='Attribute',
    yaxis_title='year',
    font=dict(family="Courier New, monospace", size=14, color="RebeccaPurple")
)

# Grafiği göster
fig.show()

<img src='https://imbalanced-learn.org/stable/_static/logo_wide.png' alt="Resim Açıklaması" width="400" height="150">

In [29]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [30]:
x = df.drop(['failure','date'], axis=1)
y = df[['failure']]

In [31]:
x=pd.get_dummies(x,drop_first=True)

In [32]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [33]:
x.head()

,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,year,month,day,device_01MC,device_01SP,device_021H,device_02L8,device_073Z,device_0AS0,device_0BNL,device_0BRH,device_0CL9,device_0D0P,device_0E6M,device_0EBC,device_0FDX,device_0FF0,device_0FFZ,device_0FZB,device_0GDQ,device_0GSG,device_0HH5,device_0JL3,device_0NDB,device_0NM6,device_0P0S,device_0PED,device_0QDG,device_0QG6,device_0QH3,device_0QL8,device_0RKC,device_0RWZ,device_0V1N,device_0VJH,device_0VP8,device_0Y8K,device_1085,device_11C6,device_11N7,device_11Q7,...,device_X5VA,device_X60T,device_X6CF,device_X6HZ,device_X6KZ,device_X6SQ,device_X6TM,device_X6V0,device_X6VJ,device_X70N,device_X711,device_X7P2,device_X7PR,device_X7QX,device_XKWR,device_XMNV,device_Y13C,device_Y2PY,device_Y4H6,device_Y6BM,device_Y8MA,device_YJ6K,device_YKBQ,device_YKEW,device_YX6Z,device_Z1W9,device_Z2JE,device_Z3FW,device_Z3G1,device_Z3KR,device_Z491,device_Z4AY,device_Z4CF,device_Z4EA,device_Z4SP,device_Z4W7,device_Z4X0,device_Z4X2,device_Z569,device_Z56F,device_Z588,device_Z5T8,device_Z62G,device_ZCPW,device_ZDTJ,device_ZG5V,device_ZJK8,device_ZK0H,device_ZVD8,device_ZVME
0,215630672,56,0,52,6,407438,0,0,7,2015,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,61370680,0,3,0,6,403174,0,0,0,2015,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,173295968,0,0,0,12,237394,0,0,0,2015,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,79694024,0,0,0,6,410186,0,0,0,2015,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,135970480,0,0,0,15,313173,0,0,3,2015,1,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [34]:
# SMOTE uygulama
smote = SMOTE(random_state=42)
x_train, y_train = smote.fit_resample(x_train, y_train)

**MODELING**

In [35]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

g=GaussianNB()
b=BernoulliNB()

In [36]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [37]:
g.fit(x_train,y_train),b.fit(x_train,y_train)

(GaussianNB(), BernoulliNB())

In [38]:
gtahmin,btahmin=g.predict(x_test),b.predict(x_test)

In [39]:
accuracy_score(gtahmin,y_test),accuracy_score(btahmin,y_test)

(0.997911562713362, 0.9992770794007791)

In [40]:
confusion_matrix(gtahmin,y_test),confusion_matrix(btahmin,y_test)

(array([[24845,    16],
        [   36,     2]]),
 array([[24881,    18],
        [    0,     0]]))

Sonuc:Bir teslimat şirketi için öngörücü bakım modeli oluşturmamız istenmişti x ve y değerlerinde tutatrsızlık mevcuttu semote kullanarak bu soruu çözdük.Arıza bakımı için eğittiğmiz model %99 oranında doğru tahmin ederek projeyi tamamladık.